In [1]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [2]:
path="../input/tabular-playground-series-nov-2022/submission_files"
li1 = os.listdir(path)
li = [path+"/"+a for a in li1]
len(li)

5000

In [22]:
datasets = []
for a in li:
    df1 = pd.read_csv(a,index_col="id")
    df1.columns=[a.split("/")[-1].split(".csv")[0]]
    datasets.append(df1)
df=pd.concat(datasets,axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Columns: 5000 entries, 0.7301891713 to 0.7222809303
dtypes: float64(5000)
memory usage: 1.5 GB


In [18]:
train=df.iloc[:20000]
test=df.iloc[20000:]
del datasets,df

In [19]:
labels = pd.read_csv("../input/tabular-playground-series-nov-2022/train_labels.csv",index_col="id")

In [20]:
params = {"boosting_type":["gbdt","dart","goss","rf"],
         "num_leaves":[5,10,25,50,100,500],
         "learning_rate":[0.1,0.01,0.05,0.001,0.005,0.2,0.3],
         "n_estimators":[50,100,200,500,1000],
         "min_child_samples":[20,50,100,500,1000,5000,10000]}

In [9]:
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV

In [10]:
model = lgb.LGBMClassifier(device="gpu")

In [11]:
clf = RandomizedSearchCV(model,params,scoring=log_loss,verbose=2,n_iter=10,cv=2)

In [13]:
clf.fit(train,labels.label.values)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END boosting_type=goss, learning_rate=0.005, min_child_samples=10000, n_estimators=100, num_leaves=500; total time=  12.7s
[CV] END boosting_type=goss, learning_rate=0.005, min_child_samples=10000, n_estimators=100, num_leaves=500; total time=   6.6s
[CV] END boosting_type=dart, learning_rate=0.3, min_child_samples=100, n_estimators=500, num_leaves=25; total time= 3.0min
[CV] END boosting_type=dart, learning_rate=0.3, min_child_samples=100, n_estimators=500, num_leaves=25; total time= 3.0min



KeyboardInterrupt



In [12]:
df = pd.DataFrame(clf.predict_proba(test))

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [15]:
df

,0,1
0,0.464870,0.535130
1,0.255882,0.744118
2,0.736080,0.263920
3,0.597267,0.402733
4,0.896531,0.103469
...,...,...
19995,0.823185,0.176815
19996,0.693718,0.306282
19997,0.409509,0.590491
19998,0.421226,0.578774


In [17]:
df1 = pd.read_csv("../input/tabular-playground-series-nov-2022/sample_submission.csv",index_col="id")
df1["pred"]=df[1].values
df1.to_csv("subNN6-5.csv")